In [2]:
import mlflow
from mlflow.tracking import MlflowClient

import os, sys

import torch
import torch.nn.functional as F

HOME = os.environ["HOME"]
CARDIAC_COMA_REPO = f"{HOME}/01_repos/CardiacCOMA/"
import os; os.chdir(CARDIAC_COMA_REPO)

from config.cli_args import overwrite_config_items
from config.load_config import load_yaml_config, to_dict

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image

import pandas as pd
import shlex
from subprocess import check_output

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

#import surgeon_pytorch
#from surgeon_pytorch import Inspect, get_layers

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

# import model.Model3D
# from utils.helpers import get_coma_args, get_lightning_module, get_datamodule
from copy import deepcopy
from pprint import pprint

from typing import List
from tqdm import tqdm

import pandas as pd

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

In [3]:
from utils.helpers import get_datamodule, get_lightning_module

In [4]:
from utils.mlflow_helpers import get_model_pretrained_weights

In [5]:
CARDIAC_COMA_REPO = "/home/rodrigo/01_repos/CardiacCOMA/"
import os; os.chdir(CARDIAC_COMA_REPO)

In [6]:
def overwrite_ref_config(ref_config, run_info):
    
    '''
    This is a workaround for adjusting the configuration of those runs that didn't have a YAML configuration file logged as an artifact.
    '''
    
    config = deepcopy(ref_config)
    config.network_architecture.latent_dim = int(run_info["params.latent_dim"])
    config.loss.regularization.weight = float(run_info["params.w_kl"])
    config.optimizer.parameters.lr = float(run_info["params.lr"])
    config.sample_sizes = [100, 100, 100, 100]
    
    return config


In [7]:
home = os.environ["HOME"]

In [8]:
cardiac_gwas_repo = f"{home}/01_repos/CardiacGWAS"
mlruns_dir = f"{home}/01_repos/CardiacCOMA/mlruns"

In [9]:
good_runs_df = pd.read_csv(f"{cardiac_gwas_repo}/good_runs.csv")

In [28]:
run_w = widgets.Select(options=good_runs_df.run_id.to_list())

@interact
def load_model(run_id=run_w):
    global _run_id, config
    _run_id = run_id
    config_file = f"{mlruns_dir}/1/{run_id}/artifacts/config.yaml"    
    config = load_yaml_config(config_file)
    config.sample_sizes = [100, 100, 100, 100]#.append(-1)
    # pprint(to_dict(config))
    return _run_id, config

interactive(children=(Select(description='run_id', options=('b6a7eedfc0e84b8b9d1f099ed5f158c4', 'e6490cf8389f4…

In [29]:
def get_model_pretrained_weights(exp_id, run_id):
    
    # run_info = runs_df.loc[experiment_id, run_id].to_dict()
    chkpt_dir = f"{CARDIAC_COMA_REPO}/mlruns/{exp_id}/{run_id}/checkpoints"
    
   # run_info["artifact_uri"].replace("file://", "")   
 
   # #TODO: modify this
   # try:
   #     chkpt_dir = os.path.join(artifact_uri, "restored_model_checkpoint")
   #     os.listdir(chkpt_dir)
   # except:
   #     chkpt_dir = os.path.join(os.path.dirname(artifact_uri), "checkpoints")
   # 
    chkpt_file = os.path.join(chkpt_dir, os.listdir(chkpt_dir)[0])
    
    model_pretrained_weights = torch.load(chkpt_file, map_location=torch.device('cpu'))["state_dict"]
    
    # Remove "model." prefix from state_dict's keys.
    _model_pretrained_weights = {k.replace("model.", ""): v for k, v in model_pretrained_weights.items()}
    # print(_model_pretrained_weights)
    return _model_pretrained_weights

In [31]:
# pl.utilities.seed.reset_seed() # seed_everything(seed=None)
# pl.utilities.seed.seed_everything(seed=None)

# config = overwrite_ref_config(ref_config, run_info)
# pprint(to_dict(config))

dm = get_datamodule(config, perform_setup=True)
model = get_lightning_module(config, dm)

exp_id = "1"
weights = get_model_pretrained_weights(exp_id, _run_id)
model.model.load_state_dict(weights)

[100, 100, 100, 100]


<All keys matched successfully>

In [ ]:
s = dm.dataset[1]['s']
s_hat = model(s)[0][0]
mse(s, s_hat)